In [1]:
import sys
import os
import py_entitymatching as em
print('magellan version:' + em.__version__)
import re
import csv
import pandas as pd

from constants import csv_headers
import blocker, custom_matcher

magellan version:0.1.0


/Users/Ken/anaconda/lib/python3.5/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
working_dir = os.path.dirname(os.getcwd())
path_to_csv_dir = working_dir + os.sep + 'csv_files'+ os.sep

# STEP 2 - MAGELLAN

### CHOOSE RANDOM FOREST ###

Below are the subsequent steps:
* Downsizing data - using whole AOM dataset with y_parameter of 3
* Blocking using the previous rule on the downsized dataset
* Applying the classifier on the blocked dataset

In [4]:
AOM = em.read_csv_metadata(path_to_csv_dir + '2cleaned_aom.csv', key = csv_headers.AOM_INDEX)
WHED = em.read_csv_metadata(path_to_csv_dir + '2cleaned_whed.csv', key = csv_headers.WHED_INDEX)




In [ ]:
# building inverted index based on B
final_WHED, final_AOM = em.down_sample(WHED, AOM, size=9532, y_param=3)

In [ ]:
em.save_table(final_AOM, path_to_csv_dir + '3downsized_AOM.pkl')
em.save_table(final_WHED, path_to_csv_dir + '3downsized_WHED.pkl')

In [ ]:
final_C = blocker.blocking(final_WHED, final_AOM, csv_headers.WHED, csv_headers.AOM)
final_C.head(n=5)

In [ ]:
em.save_table(final_C, path_to_csv_dir + '4blocked_samplespace.pkl')

### Load model and classify

In [5]:
final_C = em.load_table(path_to_csv_dir + '4blocked_samplespace.pkl', metadata_ext='.pklmetadata')
em.set_ltable(final_C, WHED)
em.set_rtable(final_C, AOM)
em.set_fk_ltable(final_C, 'ltable_' + csv_headers.WHED_INDEX)
em.set_fk_rtable(final_C, 'rtable_' + csv_headers.AOM_INDEX)
print(final_C.shape)
final_C.head()

(27887, 13)


,_id,ltable_a_id,rtable_person_id,ltable_a_name,ltable_a_city,ltable_a_prov,ltable_a_country,ltable_a_web,rtable_a_name,rtable_a_city,rtable_a_prov,rtable_a_country,rtable_a_email_server
0,0,9,54111,aalborg university,aalborg,NaN,denmark,http://www.aau.dk,aalborg university,branbrand,NaN,denmark,u-k.dk
1,1,9,54203,aalborg university,aalborg,NaN,denmark,http://www.aau.dk,aalborg university,aalborg,NaN,denmark,hum.aau.dk
2,2,10,29724,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,espoo,NaN,finland,aalto.fi
3,3,10,14261,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,espoo,NaN,finland,stratnet.org
4,4,10,43727,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,helsinki,finland,aalto.fi


In [ ]:
F = em.get_features_for_matching(WHED, AOM)
em.add_blackbox_feature(F, 'is_same_server', custom_matcher.is_same_server)
em.add_blackbox_feature(F, 'is_same_single_server', custom_matcher.is_same_single_server)


final_K = em.extract_feature_vecs(final_C, 
                                feature_table=F, 
                                show_progress=False)  
final_K = em.impute_table(final_K, 
                exclude_attrs=['_id', 'ltable_' + csv_headers.WHED_INDEX, 'rtable_' + csv_headers.AOM_INDEX],
                strategy='mean')

In [ ]:
em.save_table(final_K,'5feature_vector')
final_K.head()

In [18]:
# We have 37 features
final_K.shape  

(27887, 37)

In [ ]:
rf = em.load_object('./rf_classifier.pkl')

predictions = rf.predict(table=final_K, exclude_attrs=['_id', 'ltable_' + csv_headers.WHED_INDEX, 'rtable_' + csv_headers.AOM_INDEX],
                append=True, target_attr='predicted', inplace=False)

In [ ]:
matched = predictions[predictions['predicted'] == 1]
matched.shape
matched_id = matched['_id']
df = final_C.loc[final_C['_id'].isin(matched_id)]

# Keep American universities only
df = df[df.rtable_a_country == 'united states']
df.to_csv(path_to_csv_dir +'6matched_tuples.csv', index=False)
